In [1]:
import os

In [2]:
%pwd

's:\\OneDrive\\Documents\\Text_Summarizer_AWS_End_To_End\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
import shutil
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size 

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded! with following headers:\n{headers}")
            except Exception as e:
                logger.error(f"Error downloading the file: {e}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### Not able to unzip the files so that above function only download files
### I need to unzip the files manually as required format

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-11 00:07:31,548] INFO: common: Yaml file: config\config.yaml loaded sucessfully]
[2023-08-11 00:07:31,554] INFO: common: Yaml file: params.yaml loaded sucessfully]
[2023-08-11 00:07:31,558] INFO: common: Directory: artifacts created sucessfully]
[2023-08-11 00:07:31,558] INFO: common: Directory: artifacts/data_ingestion created sucessfully]


[2023-08-11 00:07:35,606] INFO: 3233804986: artifacts/data_ingestion/data.zip downloaded! with following headers:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 09CE:19F162:B48F2:F8C92:64D5DE34
Accept-Ranges: bytes
Date: Fri, 11 Aug 2023 07:07:33 GMT
Via: 1.1 varnish
X-Served-By: cache-del21725-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1691737652.321232,VS0,VE879
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 7abde35994f947633c5f428eae9a2f1449f59f9a
Expires: Fri, 11 Aug 2023 07:12:33 GMT
Source-Age: 1

]
